# BERT For Maked Language Model

In [1]:
$wget -q --show-progress --progress=bar:force:noscroll -O bert-base-uncased-config.json https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json
$wget -q --show-progress --progress=bar:force:noscroll -O bert-base-uncased-model.gt https://cdn.huggingface.co/bert-base-uncased-rust_model.ot

bert-base-uncased-c 100%[===================>]     433  --.-KB/s    in 0s      
bert-base-uncased-m 100%[===================>] 509.49M  4.03MB/s    in 3m 39s  


In [ ]:
import(
    "fmt"
    
    ts "github.com/sugarme/gotch/tensor"
    "github.com/sugarme/gotch/nn"
    "github.com/sugarme/gotch"
    "github.com/sugarme/tokenizer"
    "github.com/sugarme/tokenizer/pretrained"
    "github.com/sugarme/transformer"
    "github.com/sugarme/transformer/bert"
)

In [ ]:
device := gotch.CPU
vs := nn.NewVarStore(device)
// load BERT config
config, err := bert.ConfigFromFile("bert-base-uncased-config.json")
if err != nil{ fmt.Print(err)}
model := bert.NewBertForMaskedLM(vs.Root(), config)
// load weights for BERT masked language model
err := vs.Load("bert-base-uncased-model.gt")
if err != nil{fmt.Print(err)}
// load pretrained bert-base-uncased tokenizer
tk := pretrained.BertBaseUncased()

In [ ]:
// Input sample
sentence := "Remi is 6 years old and he goes to [MASK] every day."
// Encode the input
enc, err := tk.EncodeSingle(sentence, true)
if err != nil{fmt.Print(err)}
var tokInput []int64
for _, id := range enc.Ids{
    tokInput = append(tokInput, int64(id))
}
// Create input tensors from token Ids
// NOTE: BERT model is designed to take multiple samples.
tokTensors := []ts.Tensor{*ts.TensorFrom(tokInput)} 
inputTs, err := ts.Stack(tokTensors, 0)
if err != nil{fmt.Print(err)}
input, err := inputTs.To(device, true)
if err != nil { fmt.Print(err) }

// Forward through the model
var output *ts.Tensor
ts.NoGrad(func(){
    output, _, _ = model.ForwardT(input, ts.None, ts.None, ts.None, ts.None, ts.None, ts.None, false)
})

// Get first sample
output1, err := output.Get(0)
if err != nil{fmt.Print(err)}
// Get p-values for [MASK] token - NOTE: there're added special tokens [CLS] sentence [SEP]
values, err := output1.Get(11)
if err != nil{ fmt.Print(err) }
// Get the best value
am, err := values.Argmax([]int64{0}, false, false)
if err != nil{ fmt.Print(err) }
id := am.Int64Values()[0]
// Lookup token from vocab
word, ok := tk.IdToToken(int(id))
if !ok { fmt.Printf("Token not found for input Id: %v\n", id)}
fmt.Printf("Tokens: %q\n", enc.Tokens)
fmt.Printf("Input: %v - Output: %v\n", sentence, word)